In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn_pandas import DataFrameMapper
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn_pandas import gen_features

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, log_loss
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
import umap
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv("hypertension_data.csv")

NameError: name 'pd' is not defined

In [3]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

numeric_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_features = ['slope' , 'ca' , 'thal' , 'restecg' , 'cp']
bin = ['sex', 'fbs', 'exang']

numeric_transformations = gen_features(
    columns=[[col] for col in numeric_features],
    classes=[
        {'class': SimpleImputer, 'strategy': 'median'},
        {'class': StandardScaler}
    ]
)

categorical_transformations = gen_features(
    columns=[[col] for col in categorical_features],
        classes=[
        {'class': SimpleImputer, 'strategy': 'most_frequent'},
        {'class': OneHotEncoder}
    ]
)
bin_trans = gen_features(
    columns=[[col] for col in bin],
        classes=[
        {'class': SimpleImputer, 'strategy': 'median'},
    ]
)

preprocessor = DataFrameMapper(
    numeric_transformations + categorical_transformations + bin_trans,
    df_out=True, default=False
)

X_train_prepr = preprocessor.fit_transform(X_train)
X_test_prepr = preprocessor.transform(X_test)

In [1]:
X_train_prepr

NameError: name 'X_train_prepr' is not defined

# Побудувати модель зменшення вимірності PCA 

In [ ]:
pca = PCA(n_components=X_train_prepr.shape[1])
pca.fit(X_train_prepr)

X_train_pca = pca.transform(X_train_prepr)
X_test_pca = pca.transform(X_test_prepr)

Провести аналіз explained variance для кожної компоненти та кумулятивно, оцінити оптимальну кількість компонент;

In [ ]:
# Аналіз explained variance для кожної компоненти
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# Візуалізація explained variance
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.5, align='center',
        label='Explained Variance by Component')
plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid',
         label='Cumulative Explained Variance')
plt.xlabel('Principal Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance by Principal Components')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

Зменшити вимірність до двох компонент (або вибрати перші дві компоненти з вже навченого PCA) та побудувати візуалізацію об’єктів у новому просторі (колір точки повинен залежати від таргет-змінної);

In [ ]:
# Зменшення вимірності до двох компонент
pca_2d = PCA(n_components=2)
X_train_pca_2d = pca_2d.fit_transform(X_train_prepr)

# Візуалізація об'єктів у новому просторі
plt.figure(figsize=(10, 6))
plt.scatter(X_train_pca_2d[:, 0], X_train_pca_2d[:, 1], c=y_train, cmap='viridis')
plt.title('Objects in 2D PCA Space (Colored by Target Variable)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Target')
plt.show()


# Обрати дві моделі нелінійного зменшення вимірності на базі многовидів [API: 3, 4]:


In [ ]:
# Ініціалізуємо модель t-SNE з 2 компонентами
tsne = TSNE(n_components=2, random_state=42)

# Зменшення вимірності та візуалізація результату
X_train_tsne = tsne.fit_transform(X_train_prepr)
plt.figure(figsize=(10, 6))
plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], c=y_train, cmap='viridis')
plt.title('t-SNE Visualization (Colored by Target Variable)')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.colorbar(label='Target')
plt.show()

In [ ]:

# Значення параметрів perplexity, які будемо тестувати
perplexity_values = [5, 30, 50]

# Побудова та візуалізація результатів для кожного значення perplexity
plt.figure(figsize=(15, 10))
for i, perplexity in enumerate(perplexity_values, 1):
    # Ініціалізація моделі t-SNE з відповідним значенням perplexity
    tsne_model = TSNE(n_components=2, perplexity=perplexity, random_state=42)
    
    # Зменшення вимірності та візуалізація результату
    X_train_tsne = tsne_model.fit_transform(X_train_prepr)
    
    plt.subplot(2, 3, i)
    plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], c=y_train, cmap='viridis')
    plt.title(f't-SNE (perplexity={perplexity})')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.colorbar(label='Target')

plt.tight_layout()
plt.show()

In [ ]:
# Ініціалізуємо модель UMAP з 2 компонентами
umap_model = umap.UMAP(n_components=2, random_state=42)

# Зменшення вимірності та візуалізація результату
X_train_umap = umap_model.fit_transform(X_train_prepr)
plt.figure(figsize=(10, 6))
plt.scatter(X_train_umap[:, 0], X_train_umap[:, 1], c=y_train, cmap='viridis')
plt.title('UMAP Visualization (Colored by Target Variable)')
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.colorbar(label='Target')
plt.show()

In [ ]:
# Список значень n_neighbors, які ви бажаєте перевірити
n_neighbors_values = [5, 10, 15, 20]

# Побудова UMAP та візуалізація для кожного значення n_neighbors
for n_neighbors in n_neighbors_values:
    umap_model = umap.UMAP(n_components=2, n_neighbors=n_neighbors, random_state=42)
    X_train_umap = umap_model.fit_transform(X_train_prepr)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(X_train_umap[:, 0], X_train_umap[:, 1], c=y_train, cmap='viridis')
    plt.title(f'UMAP Visualization (n_neighbors={n_neighbors})')
    plt.xlabel('UMAP Component 1')
    plt.ylabel('UMAP Component 2')
    plt.colorbar(label='Target')
    plt.show()

ф

In [ ]:

# Список значень n_neighbors для UMAP та perplexity для t-SNE
umap_n_neighbors_values = [5, 10, 15, 20]
tsne_perplexity_values = [30, 50, 70, 100]

fig, axs = plt.subplots(len(umap_n_neighbors_values) + len(tsne_perplexity_values), 2, figsize=(10, 6 * (len(umap_n_neighbors_values) + len(tsne_perplexity_values))))

# Побудова UMAP та візуалізація для кожного значення n_neighbors
for i, n_neighbors in enumerate(umap_n_neighbors_values):
    umap_model = umap.UMAP(n_components=2, n_neighbors=n_neighbors, random_state=42)
    X_train_umap = umap_model.fit_transform(X_train_prepr)
    
    axs[i, 0].scatter(X_train_umap[:, 0], X_train_umap[:, 1], c=y_train, cmap='viridis')
    axs[i, 0].set_title(f'UMAP Visualization (n_neighbors={n_neighbors})')
    axs[i, 0].set_xlabel('UMAP Component 1')
    axs[i, 0].set_ylabel('UMAP Component 2')
    fig.colorbar(axs[i, 0].collections[0], ax=axs[i, 0], label='Target')

# Побудова t-SNE та візуалізація для кожного значення perplexity
for i, perplexity in enumerate(tsne_perplexity_values):
    tsne_model = TSNE(n_components=2, perplexity=perplexity, random_state=42)
    X_train_tsne = tsne_model.fit_transform(X_train_prepr)
    
    axs[i + len(umap_n_neighbors_values), 1].scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], c=y_train, cmap='viridis')
    axs[i + len(umap_n_neighbors_values), 1].set_title(f't-SNE Visualization (perplexity={perplexity})')
    axs[i + len(umap_n_neighbors_values), 1].set_xlabel('t-SNE Component 1')
    axs[i + len(umap_n_neighbors_values), 1].set_ylabel('t-SNE Component 2')
    fig.colorbar(axs[i + len(umap_n_neighbors_values), 1].collections[0], ax=axs[i + len(umap_n_neighbors_values), 1], label='Target')

plt.tight_layout()
plt.show()

# Побудувати модель бінарної класифікації (лог. регресію) зі зменшенням вимірності:


In [ ]:
# Створення пайплайну зі зменшенням вимірності PCA та логістичною регресією
pipeline = Pipeline([
    ('pca', pca),  # Використовуємо вже навчений PCA
    ('clf', LogisticRegression(random_state=42))
])

# Параметри, які будемо тюнити
parameters = {
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100]  # Регуляризаційний параметр логістичної регресії
}

# GridSearchCV для налаштування пайплайну
grid_search = GridSearchCV(pipeline, parameters, cv=5)
grid_search.fit(X_train_prepr, y_train)

# Найкращі параметри та результати
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f'Найкращі параметри: {best_params}')
print(f'Найкращий результат: {best_score}')

# Отримання оптимального пайплайну
optimal_pipeline = grid_search.best_estimator_

In [ ]:
# Отримання прогнозів на тестовому наборі
y_pred = optimal_pipeline.predict(X_test_prepr)
y_pred_proba = optimal_pipeline.predict_proba(X_test_prepr)[:, 1]

# Обчислення метрик
binary_crossentropy = log_loss(y_test, y_pred_proba)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Виведення результатів
print(f'Binary Crossentropy: {binary_crossentropy:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F₁ Score: {f1:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')